## Extract FTE data eith event locations

In [7]:
import numpy as np
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/naveen/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
# For Train Test SPlit
random_state = 3

#Load Data
segments = np.load('themis/segments-newdata-all.npy')

merged = segments.reshape(-1, segments.shape[-1])
mean = merged.mean(axis=0)
std = merged.std(axis=0)
segments = segments - mean
segments = (segments / std[None, None, :])[:,:,[5]]

min = np.min(segments)
segments = segments - min
max = np.max(segments)
segments = segments / max
labels = np.load('themis/labels-newdata-all.npy')
labels = np.asarray(pd.get_dummies(labels))
with open('themis/bbox.pickle','rb') as f:
    bboxes = pickle.load(f)

GT_len = [np.sum(bboxes[i][:,2]-bboxes[i][:,0])/bboxes[i].shape[0] for i in bboxes if bboxes[i].shape[0] > 0]
avg_GT_len = sum(GT_len)/len(GT_len)
# get Train val test split
indices = np.arange(0,segments.shape[0])
indices_tmp, indices_test = train_test_split(indices, test_size=0.2, random_state=random_state)

indices_train, indices_val  = train_test_split(indices_tmp, test_size=0.1, random_state=random_state)


In [12]:
# non empty bbox indices
indices_non_empty_train = [i for i in indices_train if bboxes[i].size is not 0]
indices_non_empty_val = [i for i in indices_val if bboxes[i].size is not 0]
indices_non_empty_test = [i for i in indices_test if bboxes[i].size is not 0 ]
pos_list = [i for i in indices if bboxes[i].size is not 0]
# TODO have a look carefully


X_train = segments[indices_train, :].reshape(len(indices_train), 1440, 1, 1)
y_train = labels[indices_train]
X_val = segments[indices_val, :].reshape(len(indices_val), 1440, 1, 1)
y_val = labels[indices_val]
X_test = segments[indices_test, :].reshape(len(indices_test), 1440, 1, 1)
y_test = labels[indices_test]
bbox_train = {i: bboxes[i] for i in indices_train}
bbox_test = {i: bboxes[i] for i in indices_test}
bbox_val = {i: bboxes[i] for i in indices_val}
